# AstroCLIP

This notebook finetunes the astroclip model on this dataset.

## Load model

In [ ]:
from astroclip.models import AstroClipModel

model = AstroClipModel.load_from_checkpoint(
    "../../OSS/AstroCLIP/pretrained/astroclip.ckpt"
).cpu()

c:\Users\mi3se\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'image_encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['image_encoder'])`.
c:\Users\mi3se\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'spectrum_encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['spectrum_encoder'])`.


Using CLIPLoss loss


## Data

In [16]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((144, 144)),
    transforms.ToTensor()
])

In [33]:
from datamodule import VLASSLoader
datamodule = VLASSLoader(
    root = "./data", batch_size=32, pin_memory=True,
    transform = transform
)

## LinearProbe

In [ ]:
from torch import nn
from models.vision import VisionModel

class AstroClip(VisionModel):
    def __init__(self, num_classes=4, embed_dim=1024, freeze=True):
        super().__init__(num_classes=num_classes)
        self.backbone = model
        self.head = nn.Linear(embed_dim, num_classes)
        
        if freeze:
            for param in self.backbone.parameters():
                param.requires_grad = False

    def forward(self, x):
        x = self.backbone(x, input_type='image')
        x = self.head(x)
        return x

In [49]:
clip_model = AstroClip().cuda()

In [50]:
from lightning import Trainer

trainer = Trainer(
    max_steps=5, 
    accelerator='gpu', devices=1
)
trainer.fit(clip_model, datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Loading data from file: ./data\vlass_data_array.p
Loading labels from file: ./data\vlass_labels.p
61351 images, each of size 64 x 64 pixels.
There are 61351 corresponding labels - one category for each image.
Loading data from file: ./data\vlass_data_array.p
Loading labels from file: ./data\vlass_labels.p
61351 images, each of size 64 x 64 pixels.
There are 61351 corresponding labels - one category for each image.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type           | Params | Mode 
----------------------------------------------------
0 | backbone | AstroClipModel | 370 M  | eval 
1 | head     | Linear         | 4.1 K  | train
----------------------------------------------------
4.1 K     Trainable params
370 M     Non-trainable params
370 M     Total params
1,483.210 Total estimated model params size (MB)
1         Modules in train mode
525       Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\mi3se\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Val size: 12271
Train size: 49080


c:\Users\mi3se\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=5` reached.


## Test

In [ ]:
val_dataloader = datamodule.val_dataloader()
clip_model = clip_model.cuda()

In [56]:
a = [1, 2, 3, 4]
a[:-2]

[1, 2]

In [55]:
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score

clip_model.eval()
with torch.no_grad():
    y_trues, y_preds = [], []
    
    for inputs, labels in tqdm(val_dataloader, total=len(val_dataloader)):
        outputs = clip_model(inputs.cuda())
        predicted = torch.argmax(outputs, 1)
        
        y_trues.extend(labels.detach().cpu().numpy())
        y_preds.extend(predicted.detach().cpu().numpy())
        
acc = accuracy_score(y_trues, y_preds)
f1 = f1_score(y_trues, y_preds, average='macro')

 17%|█▋        | 67/383 [01:38<07:46,  1.48s/it]


KeyboardInterrupt: 

In [ ]:
from utils import test_model
result = test_model(clip_model, )
print(f'Accuracy = {result["accuracy"]}, F1 = {result["f1"]}')
print('Classification report:\n', result['classification_report'])
print('Confusion Matrix:\n', result['confusion_matrix'])